# 수집된 데이터에 대한 통계 분석
* 데이터 
> 2017년 12월 1일부터 2018년 1월 1일까지 32일간 [네이버](http://www.naver.com)와 [다음](http://www.daum.net)의 랭킹뉴스와 뉴스의 댓글을 크롤링함.

In [10]:
import os
import pandas as pd 
from glob import glob
import sys
import time, re, pickle, itertools
import urllib3, json
sys.path.append('~/Documents/GitHub/Private_Project/personal_project/')
import re
from operator import itemgetter
import html
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import chat_bot as cb
import Database_Handler as dh
from datetime import datetime
import seaborn as sns
import functools
from konlpy.tag import Mecab
from ckonlpy.tag import Twitter as ctwitter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
%matplotlib inline

In [11]:
#site = 'Naver'
site = 'daum'
if site == 'daum':
    binList = []
    collection = 'newsDaum'
elif site.lower() == 'naver':
    binList = []
    collection = 'newsNaver'

mongodb = dh.ToMongoDB(*dh.AWS_MongoDB_Information())
dbname = 'hy_db'
useDb = dh.Use_Database(mongodb, dbname)
useCollection = dh.Use_Collection(useDb, collection)
collection = useCollection.find({'site':site})

In [17]:
mecab = Mecab()
def tokenize_pos2(doc):
    return ['/'.join(t) for t in mecab.pos(doc)]

In [18]:
rawdata = pd.read_csv('./data/sentiment_data/raw_data_for_sentiment.txt',header=None, encoding='utf-8')
rawdata.head()
X = list(rawdata[0])
y = np.array(list(rawdata[1]), dtype=int)
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model_mecab_log1 = Pipeline([
            ('vect', TfidfVectorizer(tokenizer=tokenize_pos2, ngram_range=(1,5))), 
            ('log', LogisticRegression()),
        ])

In [ ]:
%%time 
model_mecab_log1.fit(train_X, train_y)
print(classification_report(test_y, model_mecab_log1.predict(test_X)))

In [ ]:
pickle.dump(model_mecab_log1, open('./data/pre_data/classifier_log_classifier_by_mecab.pickled','wb'))

In [ ]:
for data in collection:
    text = data['title']+'. '+data['mainText']
    pre = model_mecab_log1.predict(text)
    pre_proba = model_mecab_log1.predict_proba(text)
    print (pre)
    print (pre_proba)
    break